# Create SUMMA-to-mizuRoute catchment remapping file
Creates a remap `.nc` file for cases where the routing catchments are different from the catchments used to run SUMMA. While mizuRoute is able to perform routing from grid-based outputs, SUMMA does not produce these and the code here is not setup to work with gridded model outputs.

**_This code assumes routing occurs at the GRU level of SUMMA outputs. SUMMA-HRU-level routing is not supported._**

In [1]:
# modules
import itertools
import pandas as pd
import netCDF4 as nc4
import geopandas as gpd
from pathlib import Path
from shutil import copyfile
import easymore.easymore as esmr
from datetime import datetime

#### Control file handling

In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_control_files')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Check if remapping is needed

In [6]:
# Get the remap flag
do_remap = read_from_control(controlFolder/controlFile,'river_basin_needs_remap')

In [7]:
# Check
if do_remap.lower() != 'yes':
    print('Active control file indicates remapping is not needed. Aborting.')
    exit()

#### Find location of hydrologic model (HM) catchment shapefile

In [8]:
# HM catchment shapefile path & name
hm_catchment_path = read_from_control(controlFolder/controlFile,'catchment_shp_path')
hm_catchment_name = read_from_control(controlFolder/controlFile,'catchment_shp_name')

In [9]:
# Specify default path if needed
if hm_catchment_path == 'default':
    hm_catchment_path = make_default_path('shapefiles/catchment') # outputs a Path()
else:
    hm_catchment_path = Path(hm_catchment_path) # make sure a user-specified path is a Path()

In [10]:
# Find the fields we're interested in
hm_shp_gru_id = read_from_control(controlFolder/controlFile,'catchment_shp_gruid')

#### Find location of routing model (RM) catchment shapefile

In [11]:
# Routing model catchment shapefile path & name
rm_catchment_path = read_from_control(controlFolder/controlFile,'river_basin_shp_path')
rm_catchment_name = read_from_control(controlFolder/controlFile,'river_basin_shp_name')

In [12]:
# Specify default path if needed
if rm_catchment_path == 'default':
    rm_catchment_path = make_default_path('shapefiles/river_basins') # outputs a Path()
else:
    rm_catchment_path = Path(rm_catchment_path) # make sure a user-specified path is a Path()

In [13]:
# Find the fields we're interested in
rm_shp_hru_id = read_from_control(controlFolder/controlFile,'river_basin_shp_rm_hruid')

#### Find where the intersection needs to go

In [14]:
# Intersected shapefile path. Name is set by EASYMORE as [prefix]_intersected_shapefile.shp
intersect_path = read_from_control(controlFolder/controlFile,'intersect_routing_path')
intersect_name = read_from_control(controlFolder/controlFile,'intersect_routing_name')

In [15]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_routing') # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

In [16]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

#### Find where the remapping file needs to go

In [17]:
# Remap .nc path and name
remap_path = read_from_control(controlFolder/controlFile,'settings_mizu_path')
remap_name = read_from_control(controlFolder/controlFile,'settings_mizu_remap')

In [18]:
# Specify default path if needed
if remap_path == 'default':
    remap_path = make_default_path('settings/mizuRoute') # outputs a Path()
else:
    remap_path = Path(remap_path) # make sure a user-specified path is a Path()

In [19]:
# Make the folder if it doesn't exist
remap_path.mkdir(parents=True, exist_ok=True)

#### Call EASYMORE to do the intersection 

In [20]:
# Load both shapefiles
hm_shape = gpd.read_file(hm_catchment_path/hm_catchment_name)
rm_shape = gpd.read_file(rm_catchment_path/rm_catchment_name)

In [21]:
# Create a EASYMORE object
esmr_caller = esmr()

In [22]:
# Project both shapes to equal area
hm_shape = hm_shape.to_crs('EPSG:6933')
rm_shape = rm_shape.to_crs('EPSG:6933')

In [23]:
# Run the intersection
intersected_shape = esmr.intersection_shp(esmr_called,rm_shape,hm_shape)

C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex_hs\lib\site-packages\candex\candex.py:893: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  pairs = gpd.GeoDataFrame(nei, columns=['idx1','idx2'], crs=df1.crs)
C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex_hs\lib\site-packages\candex\candex.py:897: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  pairs = gpd.GeoDataFrame(pairs, columns=pairs.columns, crs=df1.crs)


In [24]:
# Reproject the intersection to WSG84
intersected_shape = intersected_shape.to_crs('EPSG:4326')

In [25]:
# Save the intersection to file
intersected_shape.to_file(intersect_path/intersect_name)

C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex_hs\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


#### Pre-process the variables

In [26]:
# Define a few shorthand variables
int_rm_id = 'S_1_' + rm_shp_hru_id
int_hm_id = 'S_2_' + hm_shp_gru_id
int_weight = 'AP1N'

In [27]:
# Sort the intersected shape by RM ID first, and HM ID second. This means all info per RM ID is in consecutive rows
intersected_shape = intersected_shape.sort_values(by=[int_rm_id,int_hm_id])

In [28]:
# Routing Network HRU ID
nc_rnhruid = intersected_shape.groupby(int_rm_id).agg({int_rm_id: pd.unique}).values.astype(int)

In [29]:
# Number of Hydrologic Model elements (GRUs in SUMMA's case) per Routing Network catchment
nc_noverlaps = intersected_shape.groupby(int_rm_id).agg({int_hm_id: 'count'}).values.astype(int)

In [30]:
# Hydrologic Model GRU IDs that are associated with each part of the overlap
multi_nested_list = intersected_shape.groupby(int_rm_id).agg({int_hm_id: list}).values.tolist() # Get the data
nc_hmgruid = list(itertools.chain.from_iterable(itertools.chain.from_iterable(multi_nested_list))) # Combine 3 nested list into 1

In [31]:
# Areal weight of each HM GRU per part of the overlaps
multi_nested_list = intersected_shape.groupby(int_rm_id).agg({int_weight: list}).values.tolist() 
nc_weight = list(itertools.chain.from_iterable(itertools.chain.from_iterable(multi_nested_list))) 

#### Make the `.nc` file

In [32]:
# Find the dimension sizes
num_hru  = len(rm_shape)
num_data = len(intersected_shape)

In [33]:
# Function to create new nc variables
def create_and_fill_nc_var(ncid, var_name, var_type, dim, fill_val, fill_data, long_name, units):
    
    # Make the variable
    ncvar = ncid.createVariable(var_name, var_type, (dim,), fill_val)
    
    # Add the data
    ncvar[:] = fill_data    
    
    # Add meta data
    ncvar.long_name = long_name 
    ncvar.unit = units
    
    return    

In [40]:
# Make the netcdf file
with nc4.Dataset(remap_path/remap_name, 'w', format='NETCDF4') as ncid:
    
    # Set general attributes
    now = datetime.now()
    ncid.setncattr('Author', "Created by SUMMA workflow scripts")
    ncid.setncattr('History','Created ' + now.strftime('%Y/%m/%d %H:%M:%S'))
    ncid.setncattr('Purpose','Create a remapping .nc file for mizuRoute routing')
    
    # Define the seg and hru dimensions
    ncid.createDimension('hru', num_hru)
    ncid.createDimension('data', num_data)
    
    # --- Variables
    create_and_fill_nc_var(ncid, 'RN_hruId', 'int', 'hru', False, nc_rnhruid, \
                           'River network HRU ID', '-')
    create_and_fill_nc_var(ncid, 'nOverlaps', 'int', 'hru', False, nc_noverlaps, \
                           'Number of overlapping HM_HRUs for each RN_HRU', '-')
    create_and_fill_nc_var(ncid, 'HM_hruId', 'int', 'data', False, nc_hmgruid, \
                           'ID of overlapping HM_HRUs. Note that SUMMA calls these GRUs', '-')
    create_and_fill_nc_var(ncid, 'weight', 'f8', 'data', False, nc_weight, \
                           'Areal weight of overlapping HM_HRUs. Note that SUMMA calls these GRUs', '-')

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [35]:
# Set the log path and file name
logPath = remap_path
log_suffix = '_make_remapping_file.txt'

In [36]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [37]:
# Copy this script
thisFile = '1_remap_summa_catchments_to_routing.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [38]:
# Get current date and time
now = datetime.now()

In [39]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated remapping .nc file for Hydro model catchments to routing model catchments.']
    for txt in lines:
        file.write(txt) 